In [1]:
!nvidia-smi

Thu Jun 25 09:43:47 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 980     Off  | 00000000:01:00.0 Off |                  N/A |
| 37%   42C    P0    35W / 180W |      0MiB /  4043MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  Quadro K2200        Off  | 00000000:05:00.0 Off |                  N/A |
| 42%   40C    P8     1W /  39W |   3967MiB /  4041MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
        

0.Background  
1.Hat  
2.Hair  
3.Glove  
4.Sunglasses  
5.UpperClothes  
6.Dress  
7.Coat  
8.Socks  
9.Pants  
10.Torso-skin  
11.Scarf  
12.Skirt  
13.Face  
14.Left-arm  
15.Right-arm  
16.Left-leg  
17.Right-leg  
18.Left-shoe  
19.Right-shoe  

In [2]:
import numpy as np
import os
os.environ['TF_KERAS'] = '1'
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
import tensorflow as tf
import segmentation_models as sm
from tensorflow.keras import callbacks
from keras_radam import RAdam

import albumentations as albu

import cv2
import datetime
import matplotlib.pyplot as plt

Using TensorFlow backend.


Segmentation Models: using `keras` framework.


In [4]:
# tf.config.list_physical_devices('GPU')

In [5]:
sm.set_framework('tf.keras')

BACKBONE = 'resnet50'
BATCH_SIZE = 4

In [6]:
root_path = '.'
checkpoints_path = f'{root_path}/checkpoints'

In [7]:
num_classes = 20

## DataGenerator

In [8]:
from tensorflow.python.keras.utils.data_utils import Sequence
from keras.utils import to_categorical

In [9]:
dataset_path = '/work/develop/datasets/LIP/CIHP/instance-level_human_parsing'

train_images_path = f'{dataset_path}/Training/Images'
train_id_path = f'{dataset_path}/Training/train_id.txt'
train_categories_path = f'{dataset_path}/Training/Category_ids'

valid_images_path = f'{dataset_path}/Validation/Images'
valid_categories_path = f'{dataset_path}/Validation/Category_ids'
valid_id_path = f'{dataset_path}/Validation/val_id.txt'

In [10]:
def get_preprocessing(preprocessing_fn):
    _transform = [
        albu.Lambda(image=preprocessing_fn),
    ]
    
    return albu.Compose(_transform)

def get_train_augmentations():
    _transform = [
#         albu.VerticalFlip(), 
        albu.HorizontalFlip(), 
        albu.ShiftScaleRotate(scale_limit=0.1, rotate_limit=10, shift_limit=0.05, border_mode=0), 
        albu.RandomBrightnessContrast(),
        albu.RandomContrast(),
        # albu.GridDistortion(num_steps=2),
#         albu.Blur(blur_limit=3)
    ]
    
    return albu.Compose(_transform, p=0.5)  # probabilty of tranformation

In [11]:
class DataGenerator(Sequence):
    def __init__(self, usage, batch_size, num_classes, preprocessing=None, augmentation=None):
        self.usage = usage

        if usage == 'train':
            id_file = train_id_path
            self.images_folder = train_images_path
            self.categories_folder = train_categories_path
        else:
            id_file = valid_id_path
            self.images_folder = valid_images_path
            self.categories_folder = valid_categories_path
            
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.preprocessing = preprocessing
        self.augmentation = augmentation
        
        with open(id_file, 'r') as f:
            self.names = f.read().splitlines()

        np.random.shuffle(self.names)

    def __len__(self):
        return int(np.ceil(len(self.names) / float(self.batch_size)))

    def __getitem__(self, idx):
        i = idx * self.batch_size
        
        img_rows, img_cols = 224, 224

        length = min(self.batch_size, (len(self.names) - i))
        batch_x = np.empty((length, img_rows, img_cols, 3), dtype=np.float32)
        batch_y = np.empty((length, img_rows, img_cols, self.num_classes), dtype=np.float32)

        for i_batch in range(length):
            name = self.names[i]
            filename = os.path.join(self.images_folder, name + '.jpg')
            image = cv2.cvtColor(cv2.imread(filename), cv2.COLOR_BGR2RGB)
            image_size = image.shape[:2]
            category = self.__get_category(self.categories_folder, name)

            image = cv2.resize(image, (img_rows, img_cols))
            category = cv2.resize(category, (img_rows, img_cols))
        
            if self.augmentation:
                augmented = self.augmentation(image=image, mask=category)
                image = augmented['image']
                category = augmented['mask']
        
            if self.preprocessing:
                preprocessed = self.preprocessing(image=image)
                image = preprocessed['image']

            batch_x[i_batch, :, :, 0:3] = image
            batch_y[i_batch, :, :] = to_categorical(category, self.num_classes)

            i += 1

        return batch_x, batch_y
    
    def __get_category(self, categories_folder, name):
        filename = os.path.join(categories_folder, name + '.png')
        semantic = cv2.imread(filename, 0)
        return semantic

    def on_epoch_end(self):
        np.random.shuffle(self.names)

In [12]:
preprocess_input = sm.get_preprocessing(BACKBONE)

data_generator_train = DataGenerator('train', BATCH_SIZE, num_classes, get_preprocessing(preprocess_input), get_train_augmentations())
data_generator_valid = DataGenerator('valid', BATCH_SIZE, num_classes, get_preprocessing(preprocess_input))

## Train

In [13]:
epochs = 30
learning_rate = 0.001

In [14]:
timestamp = f'{datetime.datetime.now():%Y%m%d%H%M}'
prefix = f'{timestamp}_{learning_rate}_{BACKBONE}'

In [15]:
checkpoint_iou_train_weights = callbacks.ModelCheckpoint(filepath=f'{checkpoints_path}/{prefix}_iou_train_weights.hdf5', mode='max', monitor='iou_score', verbose=2, save_best_only=True, save_weights_only=True)
checkpoint_iou_val_weights = callbacks.ModelCheckpoint(filepath=f'{checkpoints_path}/{prefix}_iou_val_weights.hdf5', mode='max', monitor='val_iou_score', verbose=2, save_best_only=True, save_weights_only=True)
checkpoint_loss_train_weights = callbacks.ModelCheckpoint(filepath=f'{checkpoints_path}/{prefix}_loss_train_weights.hdf5', mode='min', monitor='loss', verbose=2, save_best_only=True, save_weights_only=True)
checkpoint_loss_val_weights = callbacks.ModelCheckpoint(filepath=f'{checkpoints_path}/{prefix}_loss_val_weights.hdf5', mode='min', monitor='val_loss', verbose=2, save_best_only=True, save_weights_only=True)
reduce_lr = callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, verbose=1, mode='min', min_delta=0.001, cooldown=0, min_lr=1e-5)
# early_stopping = callbacks.EarlyStopping(monitor='val_iou_score', min_delta=0.0001, patience=15, verbose=1, mode='auto', baseline=0.95, restore_best_weights=False)
early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
callbacks_list = [checkpoint_iou_train_weights, checkpoint_iou_val_weights, checkpoint_loss_train_weights, checkpoint_loss_val_weights, reduce_lr, early_stopping]

In [16]:
model = sm.Unet(BACKBONE, input_shape=(224, 224, 3), classes=num_classes, activation='softmax', encoder_weights='imagenet')
model.load_weights(f'{checkpoints_path}/202006241823_0.001_resnet50_loss_val_weights.hdf5')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [17]:
model.compile(
    optimizer=RAdam(learning_rate=learning_rate, weight_decay=0.0001, warmup_proportion=0.1, min_lr=1e-5),
    loss=sm.losses.cce_jaccard_loss,
    metrics=[sm.metrics.iou_score],
)

In [ ]:
hist = model.fit_generator(
        generator=data_generator_train,
        validation_data=data_generator_valid,
        epochs=epochs,
        callbacks=callbacks_list,
        workers=4,
        verbose=1
    )

Epoch 1/30
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1249/7070 [====>.........................] - ETA: 4:39 - loss: 0.6952 - iou_score: 0.3438
Epoch 00001: iou_score improved from -inf to 0.36512, saving model to ./checkpoints/202006250943_0.001_resnet50_iou_train_weights.hdf5

Epoch 00001: val_iou_score improved from -inf to 0.34380, saving model to ./checkpoints/202006250943_0.001_resnet50_iou_val_weights.hdf5

Epoch 00001: loss improved from inf to 0.66941, saving model to ./checkpoints/202006250943_0.001_resnet50_loss_train_weights.hdf5

Epoch 00001: val_loss improved from inf to 0.69527, saving model to ./checkpoints/202006250943_0.001_resnet50_loss_val_weights.hdf5
7070/7070 [==============================] - 1619s 229ms/step - loss: 0.6694 - iou_score: 0.3651 - val_loss: 0.6953 - val_iou_score: 0.3438
Epoch 2/30
1250/7070 [====>.........................] - ETA: 4:31 - loss: 0.6980 - iou_score: 0.3410
Epoch 00002: iou_score did 

In [ ]:
fig, (ax_loss, ax_iou) = plt.subplots(1, 2, figsize=(10, 5))

ax_loss.plot(hist.history['loss'], label='loss')
ax_loss.plot(hist.history['val_loss'], label='val_loss')
ax_loss.legend()

ax_iou.plot(hist.history['iou_score'], label='iou_score')
ax_iou.plot(hist.history['val_iou_score'], label='val_iou_score')
ax_iou.legend()